In [ ]:
import numpy as np
import h5py
import os
from scipy.ndimage import generic_filter
from mintpy.utils import readfile as timeseries_readfile

# Standardized Thresholds
T_COH_THRESH = 0.85
S_COH_THRESH = 0.85
V_STD_THRESH = 2.0
NEIGHBORHOOD_SIZE = 10      
STABILITY_DENSITY = 0.8    
WINDOW_SIZE = 10           

def create_master_reference(vel_file='velocity.h5', t_coh_file='temporalCoherence.h5', s_coh_file='avgSpatialCoh.h5'):
   
    #Load Data
    v_std, meta = timeseries_readfile.read(vel_file, datasetName='velocityStd')
    t_coh, _ = timeseries_readfile.read(t_coh_file, datasetName='temporalCoherence')
    s_coh, _ = timeseries_readfile.read(s_coh_file, datasetName='coherence')
    img_height, img_width = v_std.shape

    # Apply the Filter
    stable_mask = (t_coh >= T_COH_THRESH) & (s_coh >= S_COH_THRESH) & (v_std <= V_STD_THRESH)
    
    # Spatial Density Filtering 
    def count_stable(window): return np.sum(window)
    stable_neighbor_count = generic_filter(stable_mask.astype(int), count_stable, size=NEIGHBORHOOD_SIZE)
    min_neighbors = int(STABILITY_DENSITY * (NEIGHBORHOOD_SIZE**2))
    cluster_mask = (stable_mask == 1) & (stable_neighbor_count >= min_neighbors)
    
    # Targeted Weighting 
    masked_vstd = np.where(cluster_mask == 1, v_std, np.nan) # Ignore anything outside our stable clusters
    if np.all(np.isnan(masked_vstd)):
        print("Error: No stable clusters found. Lower your thresholds.")
        return

    # Find the absolute best pixel (Lowest StdDev) within a cluster
    min_idx = np.nanargmin(masked_vstd)
    center_y, center_x = np.unravel_index(min_idx, v_std.shape)
    
    print(f"Optimal Reference Seed: Y={center_y}, X={center_x} (StdDev: {v_std[center_y, center_x]:.4f} mm/yr)")

    # Define the local weighting window
    half_w = WINDOW_SIZE // 2
    y_min, y_max = max(0, center_y - half_w), min(img_height, center_y + half_w)
    x_min, x_max = max(0, center_x - half_w), min(img_width, center_x + half_w)

    # Calculate Inverse Variance Weights
    final_weights = np.zeros_like(v_std, dtype=np.float32)
    target_zone_vstd = v_std[y_min:y_max, x_min:x_max]
    weights_subset = 1.0 / (target_zone_vstd**2 + 1e-6)
    
    # Normalize
    weights_subset /= np.max(weights_subset)
    final_weights[y_min:y_max, x_min:x_max] = weights_subset

    # 4. Save the Mask File
    output_name = 'A_Master_Reference_Weights.h5'
    with h5py.File(output_name, 'w') as f:
        f.create_dataset('mask', data=final_weights)
        for k, v in meta.items():
            f.attrs[k] = v
            
    print(f"Success! Master reference file saved as: {output_name}")

if __name__ == '__main__':
    create_master_reference()